# Evaluate the effect of anode placement (vmPFC, bipolar)

In this notebook, we evaluate the effect of the anode placement on the F7-F8 bipolar electrodes montage targetting the ventromedial prefrontal cortex (vmPFC).

In [ ]:
%load_ext lab_black

In [ ]:
# Setup
from pathlib import Path
import os
import sys

import arviz as az
import bambi as bmb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Add utility package to path
sys.path.append(os.environ.get("BRAINWEB_TDCS_CODE_DIR", "../code"))
from brainweb_tdcs import ROIS, EXPERIMENTS
from brainweb_tdcs.study import get_experiments_for_roi
from brainweb_tdcs.plot import (
    display_side_by_side,
    plot_placement_categorical,
    plot_posterior,
)

# Set path data directory
if "BRAINWEB_TDCS_DATA_DIR" not in os.environ:
    os.environ["BRAINWEB_TDCS_DATA_DIR"] = str((Path.cwd() / "../data").resolve())

# Set the random seed
RANDOM_SEED = 1234

In [ ]:
CAPTIONS = {
    "e": "Magnitude of the\nelectric field",
    "e_r": "Magnitude of the\nnormal component of electric field",
}
LABELS = {
    "e": "$| \mathbf{e} |$ (mVm$^{-1}$)",
    "e_r": "$| \mathbf{e}_r |$ (mVm$^{-1}$)",
}
FUNC_DICT = {
    "mean": np.mean,
    "std": np.std,
    "2.5%": lambda x: np.percentile(x, 2.5),
    "97.5%": lambda x: np.percentile(x, 97.5),
}

## Define the experiment

We first select the experiment we want to process:

In [ ]:
experiment = EXPERIMENTS[4]
experiment

## Define the data

The data regarding the anode placement is already formatted. The `"p"` column contains the name of the anode placement (`"reference"`, `"anterior"`, `"central"`, `"lateral"` and `"posterior"`) while the column `"p_id"` contains an integer ranging from $0$ to $4$ included ($0$ corresponding to the reference placement).

In [ ]:
data = experiment.get_data()[["sub", "p", "p_id", "e", "e_r"]]

We can then display the results for the mangitude of the electric field $| \pmb{e} |$ and of its component normal to the cortical surface $| \pmb{e}_r |$.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
plot_placement_categorical(axs[0], "e", data, LABELS["e"], CAPTIONS["e"])
plot_placement_categorical(axs[1], "e_r", data, LABELS["e_r"], CAPTIONS["e_r"])
plt.show()
columns = ["mean", "std", "25%", "75%"]
display_side_by_side(
    [
        data.groupby("p").describe()["e"][columns],
        data.groupby("p").describe()["e_r"][columns],
    ],
    ["", ""],
)

## Define the base model

We can now model the differences between the means of the populations obtained with the four anode displacements and the reference one using the following Bayesian model:
$$
| \pmb{e} | \sim \mathcal{N}(\mu_{| \pmb{e} |}, \sigma_{| \pmb{e} |}^2), \\
| \pmb{e}_r | \sim \mathcal{N}(\mu_{| \pmb{e}_r |}, \sigma_{| \pmb{e}_r |}^2),
$$
where
$$
\mu_{| \pmb{e} |} = \alpha_{| \pmb{e} |} + \sum_{p=1}^4 \beta_{| \pmb{e} |, p} \cdot X_p + \varepsilon_{| \pmb{e} |}, \\
\mu_{| \pmb{e}_r |} = \alpha_{| \pmb{e}_r |} + \sum_{p=1}^4 \beta_{| \pmb{e}_r |, p} \cdot X_p + \varepsilon_{| \pmb{e}_r |},
$$

The values of $X_p$ are either $1$ if the value in `"p_id"` is equal to $p$ or $0$. This way, the reference population is described by $\alpha + \varepsilon$ and the values of each $\beta_p$ corresponds to the difference between the reference population and the measurements resulting from the $p$-th anode displacement. If $\beta_p \approx 0$, the anode displacement does not affect the metric.

## Pooled model

The first models we fit are fully pooled. This means that we do not account for any hierarchy in the data and instead consider all the samples as independent.

These models are exactly the ones described above.

### Fit the models

Weakly informative priors are set automatically for $\alpha$, $\beta$ and $\sigma$.

In [ ]:
model_pooled_e = bmb.Model(f"e ~ C(p_id)", data)
model_pooled_e_r = bmb.Model(f"e_r ~ C(p_id)", data)

In [ ]:
trace_pooled_e = model_pooled_e.fit(draws=1000, chains=4, random_seed=RANDOM_SEED)
trace_pooled_e_r = model_pooled_e_r.fit(draws=1000, chains=4, random_seed=RANDOM_SEED)

### Results

We display the probability density of $\beta_p$ for both models as well as their $95$% HDI.

In [ ]:
idx = ["alpha", *[f"beta_{p}" for p in data["p"].unique()[1:]], "sigma"]
summary_pooled_e = az.summary(trace_pooled_e, stat_funcs=FUNC_DICT, extend=False)
summary_pooled_e_r = az.summary(trace_pooled_e_r, stat_funcs=FUNC_DICT, extend=False)
summary_pooled_e.index, summary_pooled_e_r.index = idx, idx

In [ ]:
l = len(idx[1:-1])
fig, axs = plt.subplots(4, 2, figsize=(10, 10), sharex="col")
for i, n in enumerate(idx[1:-1]):
    suffix = n.split("_")[1]
    plot_posterior(
        axs[i][0],
        trace_pooled_e,
        summary_pooled_e,
        "C(p_id)",
        LABELS["e"] if i == l - 1 else "",
        CAPTIONS["e"] if i == 0 else "",
        i,
        summary_param=n,
        show_zero=True,
        beta_suffix=suffix,
    )
    plot_posterior(
        axs[i][1],
        trace_pooled_e_r,
        summary_pooled_e_r,
        "C(p_id)",
        LABELS["e_r"] if i == l - 1 else "",
        CAPTIONS["e_r"] if i == 0 else "",
        i,
        summary_param=n,
        show_zero=True,
        beta_suffix=suffix,
    )
fig.tight_layout()
plt.show()
display_side_by_side([summary_pooled_e, summary_pooled_e_r], ["", ""])

## Hierarchical model

Now, we consider the fact that multiple records were obtained with the same subject.

To account for this, the definitions of the intercept $\alpha$ and the slopes $\beta_p$ become
$$
\alpha_i = \alpha_\text{common} + \alpha_{\text{sub}_i}, \\
\beta_{p, i} = \beta_{\text{common}, p} + \beta_{\text{sub}_i, p}.
$$

Here, we differenciate the common value of the intercept $\alpha_\text{common}$ and of the slopes $\beta_{\text{common}, p}$ from the subject specific contributions $\alpha_{\text{sub}_i}$ and $\beta_{\text{sub}_i, p}$. These contributions are modelled as centered normal distributions
$$
\alpha_\text{sub} \sim \mathcal{N}(0, \sigma_{\alpha_\text{sub}}), \\
\beta_{\text{sub}, p} \sim \mathcal{N}(0, \sigma_{\beta_\text{sub}, p}).
$$

Hence, the models become
$$
| \pmb{e} | \sim \mathcal{N}(\mu_{| \pmb{e} |, i}, \sigma_{| \pmb{e} |}^2), \\
| \pmb{e}_r | \sim \mathcal{N}(\mu_{| \pmb{e}_r |, i}, \sigma_{| \pmb{e}_r |}^2),
$$
where
$$
\mu_{| \pmb{e} |} = \alpha_{| \pmb{e} |, i} + \sum_{p=1}^4 \beta_{| \pmb{e} |, p, i} \cdot X_p + \varepsilon_{| \pmb{e} |}, \\
\mu_{| \pmb{e}_r |} = \alpha_{| \pmb{e}_r |, i} + \sum_{p=1}^4 \beta_{| \pmb{e}_r |, p, i} \cdot X_p + \varepsilon_{| \pmb{e}_r |},
$$

### Fit the models

Once again, weakly informative priors are automatically set for all the parameters.

In [ ]:
model_hierarchic_e = bmb.Model(f"e ~ C(p_id) + (C(p_id) | sub)", data)
model_hierarchic_e_r = bmb.Model(f"e_r ~ C(p_id) + (C(p_id) | sub)", data)

In [ ]:
trace_hierarchic_e = model_hierarchic_e.fit(
    draws=1000, chains=4, random_seed=RANDOM_SEED, target_accept=0.9
)
trace_hierarchic_e_r = model_hierarchic_e_r.fit(
    draws=1000, chains=4, random_seed=RANDOM_SEED, target_accept=0.9
)

### Results

Again, we display the posteriors of $\beta_p$ and their $95$% HDI for the two metrics.

In [ ]:
idx = [
    "alpha",
    *[f"beta_{p}" for p in data["p"].unique()[1:]],
    "sigma(alpha_sub)",
    *[f"alpha_sub_{i}" for i in range(len(data["sub"].unique()))],
    *[f"sigma(beta_{p}_sub)" for p in data["p"].unique()[1:]],
    *[
        f"beta_{p}_{i}"
        for p in data["p"].unique()[1:]
        for i in range(len(data["sub"].unique()))
    ],
    "sigma",
]
summary_hierarchic_e = az.summary(
    trace_hierarchic_e, stat_funcs=FUNC_DICT, extend=False
)
summary_hierarchic_e_r = az.summary(
    trace_hierarchic_e_r, stat_funcs=FUNC_DICT, extend=False
)
summary_hierarchic_e.index, summary_hierarchic_e_r.index = idx, idx

In [ ]:
l = len(idx[1:5])
fig, axs = plt.subplots(4, 2, figsize=(10, 10), sharex="col")
for i, n in enumerate(idx[1:5]):
    suffix = n.split("_")[1]
    plot_posterior(
        axs[i][0],
        trace_hierarchic_e,
        summary_hierarchic_e,
        "C(p_id)",
        LABELS["e"] if i == l - 1 else "",
        CAPTIONS["e"] if i == 0 else "",
        i,
        summary_param=n,
        show_zero=True,
        beta_suffix=suffix,
    )
    plot_posterior(
        axs[i][1],
        trace_hierarchic_e_r,
        summary_hierarchic_e_r,
        "C(p_id)",
        LABELS["e_r"] if i == l - 1 else "",
        CAPTIONS["e_r"] if i == 0 else "",
        i,
        summary_param=n,
        show_zero=True,
        beta_suffix=suffix,
    )
fig.tight_layout()
plt.show()
rows = [
    "alpha",
    "sigma(alpha_sub)",
    "beta_anterior",
    "sigma(beta_anterior_sub)",
    "beta_central",
    "sigma(beta_central_sub)",
    "beta_lateral",
    "sigma(beta_lateral_sub)",
    "beta_posterior",
    "sigma(beta_posterior_sub)",
    "sigma",
]
display_side_by_side(
    [summary_hierarchic_e.loc[rows, :], summary_hierarchic_e_r.loc[rows, :]], ["", ""]
)